In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

# APIs - Práctica independiente


#### En esta práctica independiente vamos a utilizar la práctica 3 (Mercadolibre) para generar un dataset y efectuar una predicción. 

Para realizar esta práctica, ejecuten la tercera práctica guiada, utilizando una búsqueda que les resulte de interés para generar el DataFrame. 

Pueden elegir como target regresionar el precio de la publicación, o la cantidad de unidades vendidas, y usar el target que no seleccionen como feature en su modelo.

**NOTA:** No pidan más de 10000 datos totales para no tener conflictos con los límites de la API, no pidan menos de 500 para poder modelar de forma sencilla.

### 1. Importamos los datos

Vamos a usar esta celda con un input para poder ingresar manualmente la búsqueda que hicimos en la práctica anterior. La búsqueda puede ser cualquier producto que les resulte interesante, sólo recuerden usar el mismo que usaron para generar el dataset!

Ejemplo: iphone 11

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split

from base import ColumnSelector

In [ ]:
# Lectura del DataFrame
search = input("Inserten la búsqueda utilizada para generar el DataFrame:")
df = pd.read_csv(f"../Data/{search.lower().replace(' ', '_')}_meli.csv")

### 2. Identificamos columnas numéricas y categóricas

El objetivo de este ejercicio es seleccionar columnas categóricas y numéricas. Vamos a identificarlas, de manera de poder armar un preprocesamiento acorde para cada una. **BONUS:** Identificar columnas de texto y pensar una solución que incluya text mining para nuestro pipeline final.

Antes de empezar, separemos la variable target, para evitar confusiones. Si en el histograma del ejercicio anterior vimos algún valor extremo, es un buen momento para eliminarlo.

In [ ]:
# Eliminamos la mitad del percentil superior de precio.
df = df.query("price <= @df.price.quantile(.995)")  # Si bien los iPhones son costosos,
                                                    # 10 millones de pesos parece excesivo.

In [ ]:
y = df.price
X = df.drop("price", axis=1)

In [ ]:
X.dtypes

Vamos a eliminar columnas que no parecen tener demasiada información, pero que escaparon a nuestra limpieza preliminar anterior. Si su búsqueda **no** tiene estas columnas, esta celda simplemente imprimirá un mensaje. Si alguna de ellas les parece importante en el contexto de su modelo, pueden simplemente removerlas de la lista `cols_to_remove`.

In [ ]:
cols_to_remove = ["id", "listing_type_id", "seller__real_estate_agency", "shipping__logistic_type", "magical_properties"]

for col in cols_to_remove:
    try:
        X = X.drop(col, axis=1)
        print(f"La columna {col} fue removida exitosamente.")
    except:
        print(f"La columna {col} no se encuentra en el DataFrame.")

A continuación removeremos las columnas de tags, que deberían tratarse de manera especial, así como la de Título, que es de texto. Esto **NO** es lo más correcto para mejorar la predicción, se hará simplemente para poder iterar soluciones más rápidamente. Sugerimos enfáticamente que las agreguen a su solución final.

In [ ]:
text_cols = ["title", *[col for col in df.columns if "tags" in col]]
for col in text_cols:
    try:
        X = X.drop(col, axis=1)
        print(f"La columna {col} fue removida exitosamente.")
    except:
        print(f"La columna {col} no se encuentra en el DataFrame.")

In [ ]:
X.loc[:, X.dtypes == "object"].head()

Todas las columnas de tipo object parecen ser categóricas. Revisemos los Integers.

In [ ]:
X.loc[:, X.dtypes == "int"].head()

seller__id es una columna int, pero que claramente alude a una categoría. Vamos a intentar determinar si vale la pena convertirla en variable categórica:

In [ ]:
X.seller__id.nunique() / len(X)

Parece que tenemos varios vendedores repetidos. Tenemos muchas variables de vendedor para poder prescindir del id, con lo que, si bien la marca puede ser importante, confiaremos en que esté latente en la información numérica del vendedor.

**BONUS:** intentar determinar si esta decisión es correcta.

In [ ]:
categ_cols = X.loc[:, X.dtypes == "object"].columns
num_cols = X.loc[:, (X.dtypes == "int") | (X.dtypes == "float")].columns
# Elimino la columna si sobrevive chequeos previos, si no, no hay problema.
num_cols = [col for col in num_cols if col != "seller__id"]

print("Columnas Categóricas:", categ_cols)
print("Columnas Numéricas:", num_cols)

Armemos ahora nuestras particiones de train y test, para poder trabajar más sencillamente a la hora de probar estimadores.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### 3. Planteamos la FeatureUnion para nuestras distintas variables

Armemos un Pipeline de preprocesamiento para cada tipo de variable, y usemos FeatureUnion para unir los resultados.

#### Pipeline de variables Numéricas

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
pipe_num = Pipeline(
    [
        ("select", ColumnSelector(num_cols)),
        ("impute", SimpleImputer()),  # Por default imputa con media.
        ("scale", StandardScaler())
    ]
)

#### Pipeline de variables Categóricas

In [ ]:
pipe_cat = Pipeline(
    [
        ("select", ColumnSelector(categ_cols)),
        ("impute", SimpleImputer(strategy="most_frequent")),  # Utilizo la moda como estrategia.
        ("ohe", OneHotEncoder(handle_unknown='ignore'))
    ]
)

In [ ]:
union = FeatureUnion(
    [
        ("numeric", pipe_num),
        ("categoric", pipe_cat),
        # OPCIONAL: "tag_features" y "text_features"
    ]
)

### 4. Armamos nuestro Pipeline

Generemos un Pipeline que tome como paso el FeatureUnion del ejercicio anterior.

In [ ]:
full_pipe = Pipeline(
    [
        ("preprocessing", union),
        ("rgr", None)  # Paso previsto para nuestros posibles regresores.
    ]
)

### 5. Utilizamos Gridsearch para tunear hiperparámetros
Vamos a terminar esta práctica haciendo un tuneo para el Pipeline final. Intentemos maximizar nuestro $R^2$!

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet

In [ ]:
param_grid = [
    {"rgr": [ElasticNet()], "rgr__alpha": [np.logspace(-3, 2, 7)], "rgr__l1_ratio": [0.2, 0.5, 0.9]},
    {
        "rgr": [RandomForestRegressor()],
        "rgr__n_estimators": [200, 500],
        "rgr__min_samples_leaf": np.arange(1, 11, 2),
        "rgr__max_depth": [None, 1, 2, 3]
    },
]

In [ ]:
# Puede llevar un par de minutos.
grid = GridSearchCV(full_pipe, param_grid, scoring="r2", n_jobs=-1, verbose=10)
grid.fit(X_train, y_train)

Veamos los resultados de nuestro mejor modelo:

In [ ]:
print(grid.best_score_)
grid.best_params_

#### Con estas columnas, este parece ser el mejor resultado que podemos lograr. Qué podríamos hacer para mejorarlo?